# Pandas Project - cleaning shark attacks csv

## Workflow determination
1. Determine steps to be taken
2. Set up csv in pandas and create copy to work with
3. Declare helper functions 
3. Gather information about shape and characteristics of dataframe
6. Check data types
4. Inspect data
9. Check null values
5. Change column names
8. Remove duplicates
10. Remove irrelevant columns
11. 

   * Convert dates
   * Correct data types

## Set up 


First we import the necessary libraries

In [111]:
import re

import numpy as np
import pandas as pd

from string import punctuation
from datetime import datetime

Now we declare helper functions

In [222]:
def clean_col_names(df):
    """Take a dataframe and return sanitized column names."""

    table = str.maketrans(dict.fromkeys(punctuation))
    return [c.lower()
            .strip()
            .translate(table)
            .replace(" ", "_") for c in df.columns]


def cols_missing_value(df):
    """Return fraction of column missing value."""
    
    return ((df.isna().sum())[df.isna().sum() > 0] / len(df)) * 100

def invalid_year(date):
    """Return NaT if not valid year."""
    
    # Doesn't work because of 20 non-cooperating rows
    if date == date:
        if date > datetime.strptime('2020', '%y'):
            return pd.NaT
    return date

def search_and_set(df, col, word, regex=False, lst=[]):
    """Search for term and set correspending rows to value."""
    
    # If lst is set, we use every term therein, otherwise just 
    # set word as first item
        
    if not lst:
        lst = [word]
        
    if regex:
        for item in lst:
            df.loc[df[col].str.contains(
                regex, case=False, na=False, regex=True), col] = word
    else:
        for item in lst:
            df.loc[df[col].str.contains(
                item, case=False, na=False), col] = word


Next we import the csv into pandas and inspect the head

In [113]:
df_original = pd.read_csv("attacks.csv", encoding="cp1252")
df = df_original.copy()
df.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN


## Inspect

In [114]:
# rows and columns
df.shape

(25723, 24)

In [115]:
# Types of columns
df.dtypes

Case Number                object
Date                       object
Year                      float64
Type                       object
Country                    object
Area                       object
Location                   object
Activity                   object
Name                       object
Sex                        object
Age                        object
Injury                     object
Fatal (Y/N)                object
Time                       object
Species                    object
Investigator or Source     object
pdf                        object
href formula               object
href                       object
Case Number.1              object
Case Number.2              object
original order            float64
Unnamed: 22                object
Unnamed: 23                object
dtype: object

In [116]:
# Random sample of data
df.sample(20)

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
7678,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20414,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18147,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7479,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16459,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12495,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3540,1966.12.31,31-Dec-1966,1966.0,Unprovoked,AUSTRALIA,Tasmania,"Piccaninny Point, 10 miles from Bicheno",Fishing,Alf Bosworth,M,...,2.1 m [7'] shark,"J. Green, p.35; Sun (Sydney), 1/3/1967",1966.12.31-Bosworth.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1966.12.31,1966.12.31,2763.0,NaN,NaN
10838,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8968,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20241,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [117]:
# Columns with NA values
cols_missing_value(df)

Case Number               66.170353
Date                      75.500525
Year                      75.508300
Type                      75.516075
Country                   75.694903
Area                      77.269370
Location                  77.599813
Activity                  77.615364
Name                      76.316915
Sex                       77.697003
Age                       86.506240
Injury                    75.609377
Fatal (Y/N)               77.595926
Time                      88.539439
Species                   86.533453
Investigator or Source    75.566614
pdf                       75.500525
href formula              75.504412
href                      75.500525
Case Number.1             75.500525
Case Number.2             75.500525
original order            75.473312
Unnamed: 22               99.996112
Unnamed: 23               99.992225
dtype: float64

### A few observations
   - column names contain punctuation, spaces and capitals
   - case number seems to be same as Date, and Year is a pary of previous two
   - many rows have no data at all
   - pdf is part of href formula, which in turn is equal to href
   - last 2 columns have no name and out of 25723 only 1 or 2 rows have a value
   - age and species values are mainly missing
   

## General cleanup

Now we know a little about our data set, we are going to start cleaning. First we tackle the big stuff. 

In [118]:
# Clean up column names
df.columns = clean_col_names(df)
df.columns

Index(['case_number', 'date', 'year', 'type', 'country', 'area', 'location',
       'activity', 'name', 'sex', 'age', 'injury', 'fatal_yn', 'time',
       'species', 'investigator_or_source', 'pdf', 'href_formula', 'href',
       'case_number1', 'case_number2', 'original_order', 'unnamed_22',
       'unnamed_23'],
      dtype='object')

In [119]:
# Remove duplicate rows
df.drop_duplicates(inplace=True)


In [120]:
cols_missing_value(df)

case_number                0.031686
date                       0.158428
year                       0.190114
type                       0.221800
country                    0.950570
area                       7.366920
location                   8.713561
activity                   8.776933
name                       3.485425
sex                        9.109632
age                       45.009506
injury                     0.602028
fatal_yn                   8.697719
time                      53.295311
species                   45.120406
investigator_or_source     0.427757
pdf                        0.158428
href_formula               0.174271
href                       0.158428
case_number1               0.158428
case_number2               0.158428
original_order             0.047529
unnamed_22                99.984157
unnamed_23                99.968314
dtype: float64

In [121]:
# We check the values for 'type'
df["type"].value_counts(dropna=False)

Unprovoked      4595
Provoked         574
Invalid          547
Sea Disaster     239
Boating          203
Boat             137
NaN               14
Questionable       2
Boatomg            1
Name: type, dtype: int64

In [122]:
# Change invalid and questionable to NaN
df["type"].replace(["Invalid", "Questionable"], np.nan, inplace=True)

# And merge boating
df["type"].replace(["Boat", "Boatomg"], "Boating", inplace=True)

df["type"].value_counts(dropna=False)

Unprovoked      4595
Provoked         574
NaN              563
Boating          341
Sea Disaster     239
Name: type, dtype: int64

In [123]:
# Change date to datetime column
df.date = pd.to_datetime(df["date"], errors="coerce")
df.head()

,case_number,date,year,type,country,area,location,activity,name,sex,...,species,investigator_or_source,pdf,href_formula,href,case_number1,case_number2,original_order,unnamed_22,unnamed_23
0,2018.06.25,2018-06-25,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,2018-06-18,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,2018-06-09,2018.0,NaN,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,2018-06-08,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,2018-06-04,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN


In [124]:
# Get rid of future dates. Doesn't work because of 20 columns
#df["date"] = df["date"].apply(invalid_year)


Since case_number is same as date, and original_order is a unique index, we will drop all 3 case_number columns. Also, the href_formula and pdf column are duplicate to href. Finally we remove the redundant unnamed columns

In [125]:
to_drop = ["case_number", "year", "pdf", 
           "href_formula", "case_number1", 
           "case_number2", "unnamed_22", "unnamed_23"]
df = df.drop(axis = 1, columns = to_drop)
df.head()

,date,type,country,area,location,activity,name,sex,age,injury,fatal_yn,time,species,investigator_or_source,href,original_order
0,2018-06-25,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF",http://sharkattackfile.net/spreadsheets/pdf_di...,6303.0
1,2018-06-18,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",http://sharkattackfile.net/spreadsheets/pdf_di...,6302.0
2,2018-06-09,NaN,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com",http://sharkattackfile.net/spreadsheets/pdf_di...,6301.0
3,2018-06-08,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF",http://sharkattackfile.net/spreadsheets/pdf_di...,6300.0
4,2018-06-04,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper,http://sharkattackfile.net/spreadsheets/pdf_di...,6299.0


In [126]:
# If original_order is unique then we can use it as an index

df["original_order"].value_counts(dropna=False)


NaN       3
569.0     2
799.0     1
351.0     1
5404.0    1
         ..
4424.0    1
4620.0    1
795.0     1
2836.0    1
6272.0    1
Name: original_order, Length: 6309, dtype: int64

In [127]:
df[df.original_order.isna()]

,date,type,country,area,location,activity,name,sex,age,injury,fatal_yn,time,species,investigator_or_source,href,original_order
6309,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8702,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25722,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [128]:
df["original_order"].dropna(inplace=True)
df["original_order"].value_counts(dropna=False)

569.0     2
4603.0    1
4899.0    1
810.0     1
796.0     1
         ..
3508.0    1
3256.0    1
3106.0    1
3080.0    1
6272.0    1
Name: original_order, Length: 6308, dtype: int64

In [129]:
# We change the column dtype to int
df["original_order"] = df["original_order"].astype(int)
df["original_order"]

0        6303.0
1        6302.0
2        6301.0
3        6300.0
4        6299.0
          ...  
6307     6309.0
6308     6310.0
6309        NaN
8702        NaN
25722       NaN
Name: original_order, Length: 6312, dtype: float64

In [130]:
# There are 2 rows with number 569. The highest value is 6303, so we change one of the 2 to 6304.
df[(df["original_order"] == 569) & (df["type"] == "Unprovoked")] = 6304

In [131]:
# Now the original_order is unique, we can use it as an index
df.set_index("original_order", inplace=True)

In [132]:
# If there is no location we don't want to keep the row
cols_missing_value(df)

date                      13.529785
type                       8.919518
country                    0.950570
area                       7.366920
location                   8.713561
activity                   8.776933
name                       3.485425
sex                        9.109632
age                       44.993663
injury                     0.602028
fatal_yn                   8.697719
time                      53.279468
species                   45.104563
investigator_or_source     0.427757
href                       0.158428
dtype: float64

In [133]:
# There are still many rows with very little data. We will drop all that have less than 2 columns of data
df.dropna(thresh=2, inplace=True)
df.dropna(subset=["country"], inplace=True)

In [134]:
df.sample(30)

,date,type,country,area,location,activity,name,sex,age,injury,fatal_yn,time,species,investigator_or_source,href
original_order,,,,,,,,,,,,,,,
4045.0,1997-02-28 00:00:00,Unprovoked,AUSTRALIA,Queensland,"Carrara, Nerang River",Swimming,Joanna Salmon,F,21,Left leg biten,N,NaN,a small shark,"The Advertiser, 3/1/1997, p.12",http://sharkattackfile.net/spreadsheets/pdf_di...
1251.0,1930-01-16 00:00:00,Unprovoked,SOUTH AFRICA,Western Cape Province,"Melkbaai, False Bay",Swimming,Servy LeRoux,M,23,Torso & arm bitten,N,17h30,"White shark, 4.5 m [14'9""], identity confirmed...","A. LeRoux, M. Levine, GSAF; L. Green; G.Wilson",http://sharkattackfile.net/spreadsheets/pdf_di...
2634.0,1964-08-03 00:00:00,Unprovoked,JAPAN,Okayama Prefecture,Saidaiji,Swimming,Yoshio Ukita,M,12,"Leg bitten, surgically amputated",N,NaN,NaN,K. Nakaya,http://sharkattackfile.net/spreadsheets/pdf_di...
121.0,NaT,Unprovoked,KENYA,Mombasa,Kilindini,Diving,Conway Plough & Dr. Jonathan Higgs,M,NaN,Conway's leg was bitten Higgs injury was FATAL,N,NaN,NaN,A.J. Venter,http://sharkattackfile.net/spreadsheets/pdf_di...
5795.0,2014-08-28 00:00:00,Provoked,USA,Maryland,Assateague National Seashore,Fishing,Mathew Vickers,M,33,Lacerations to foot by hooked shark PROVOKED I...,N,NaN,NaN,"Delmarva Now, 8/29/2014",http://sharkattackfile.net/spreadsheets/pdf_di...
1707.0,1946-07-18 00:00:00,Unprovoked,IRAN,Khuzestan Province,"Dorquain, on the Karun River","Fishing, probably with a net","Khodadad, (male)",M,12,Radius & ulna bared,N,NaN,NaN,"A. Anderson, M.D. / Lt. Col. R.S. Hunt, Royal ...",http://sharkattackfile.net/spreadsheets/pdf_di...
4753.0,2005-09-07 00:00:00,Unprovoked,AUSTRALIA,New South Wales,"Park Beach, Coff's Harbour",Standing,Blake Garnett,M,15,Left foot & ankle lacerated,N,Dusk,NaN,"Sydney Morning Herald, 9/9/2005",http://sharkattackfile.net/spreadsheets/pdf_di...
6210.0,2017-09-02 00:00:00,Unprovoked,USA,Florida,"Marathon, Monroe County",Swimming,Ervin Maccarty,M,NaN,Lacerations to abdomen,N,NaN,Nurse shark,NewsFlare,http://sharkattackfile.net/spreadsheets/pdf_di...
1518.0,1939-11-11 00:00:00,Provoked,AUSTRALIA,New South Wales,Maroubra,NaN,boat,NaN,NaN,Boat bitten by gaffed shark PROVOKED INCIDENT,N,NaN,whaler shark,"G.P. Whitley, p.264",http://sharkattackfile.net/spreadsheets/pdf_di...


In [135]:
df.activity.value_counts()

Surfing                                                                          970
Swimming                                                                         862
Fishing                                                                          431
Spearfishing                                                                     332
Bathing                                                                          160
                                                                                ... 
Netting pilchards                                                                  1
Treading water while alongside capsized yacht                                      1
Tech diving                                                                        1
Attempting to net shark in shark channel                                           1
Canoe capsized with 10 occupants, 8 survived,  Hamilton swam off to seek help      1
Name: activity, Length: 1513, dtype: int64

In [147]:
# Search and equalize common values in activity column

activities = ["diving", , "bathing", "paddling", "rowing",  
              "sailing", "snorkeling"]
for item in activities:
    search_and_set(df, "activity", item)

In [149]:
# Let's check what values we have in this column
df.activity.unique()

array(['paddling', 'Standing', 'surfing', 'diving', 'swimming', 'fishing',
       'Walking', 'Feeding sharks', 'Boogie boarding', 'Paddle-skiing',
       'Body boarding', 'Stand-Up Paddleboarding', 'Wading',
       'Cleaning fish', 'snorkeling', nan, '2 boats capsized', 'bathing',
       'kayaking', 'canoeing', 'SUP', 'Skimboarding', 'Touching a shark',
       'Attempting to lasso a shark', 'Photo shoot', 'Paddle boarding',
       'Kakaying', 'Washing hands', 'Grabbing shark for a selfie',
       'Tagging sharks', 'Surf skiing ', 'Floating', 'Surfng',
       'SUP Foil boarding', 'Lobstering', 'Floating in tube',
       'Teasing a shark', 'Surf-skiing', 'Feeding stingrays?',
       'Sea disaster', 'Photographing fish', 'Treading water',
       'Body boarding ', 'Playing in the water', 'Body Boarding',
       'Playing', 'Attempting to rescue a shark',
       'Photographing the shark', 'Hand feeding sharks',
       'Sitting in the water', 'rowing', 'Filming',
       'Standing in inner tub

In [254]:
# Create a dict to store multiple search terms for one item
mult_term = {}

# Now to use regex and combine more difficult terms
regex_dict = {"shipwreck": ["wreck", "sea disaster", "battle", 
                            "sunk", "sank", "capsiz", "swamped",
                            "burning", "Fell", "adrift", "washed"],
              "beach activity": ["standing", "wading", "walking", 
                                 "gathering", "lying", "splash", "playing",
                                 "wading", "inflat"],
              "shark handling": ["shark"],
              "surfing": ["surfboard", "surfing", "kite"],
              "aircrash": ["airc", "air disaster", "air/sea disaster", 
                           "airlin", "air.*force", "plane"], 
              "canoeing": [r"surf.*skiing", "canoe", "kayak"],
              "fishing": [r"fish\W", "hunting", "spear"],
              "washing": ["wash[^e]"],
              "swimming": ["swimming", "treading", "dived", "swim\W"]}

pattern = r"wreck"
df.loc[df["activity"].str.contains(pattern, case=False, na=False, regex=True)].activity
# activities, mult_term, regex_dict


original_order
4314.0                                          Shipwrecked
3285.0    16' catamaran capsized previous night, occupan...
3134.0                                            Shipwreck
2935.0    Wreck of the 1689-ton Portuguese  coaster Angoche
2926.0                                            Shipwreck
2851.0                              Wreck of the Storm King
2724.0                                            Shipwreck
2715.0          wreck of the State Oil Company ship Permina
1873.0    Shipwrecked; adrift on raft for 2 days & 2 nights
1689.0                        Bitten after dhow shipwrecked
1436.0                                    Wreck of a sampam
1241.0          Sea Disaster, wreck of the  SS Norwich City
1025.0    Steamer Una wrecked with 75 laborers onboard. ...
1020.0                            Wreck of the tug Magellan
972.0     Copra vessel with 19 on board was wrecked in a...
906.0                         Shipwrecked pearling schooner
883.0     1446-ton Norweg